# Speak with your Graph
### Neo4j Graph
The Neo4j Graph integration is a wrapper for the Neo4j Python driver. It allows querying and updating the Neo4j database in a simplified manner from LangChain. Many integrations allow you to use the Neo4j Graph as a source of data for LangChain.

### CypherQAChain
The CypherQAChain is a LangChain component that allows you to interact with a Neo4j graph database in natural language. Using an LLM and the graph schema it translates the user question into a Cypher query, executes it against the graph and uses the returned context information and the original question with a second LLM to generate a natural language response.

- https://api.python.langchain.com/en/latest/chains/langchain.chains.graph_qa.cypher.GraphCypherQAChain.html

# Neo4jVector
The Neo4j Vector integration supports a number of operations

- create vector from langchain documents
- query vector
- query vector with additional graph retrieval Cypher query
- construct vector instance from existing graph data
- hybrid search
- metadata filtering

### INstall exampe

```
pip install langchain langchain-community langchain-neo4j
pip install langchain-openai tiktoken
pip install neo4j
```

# MCP Model Context Protocol
https://www.anthropic.com/news/model-context-protocol

### https://modelcontextprotocol.io/introduction
The Model Context Protocol allows applications to provide context for LLMs in a standardized way, separating the concerns of providing context from the actual LLM interaction. This Python SDK implements the full MCP specification, making it easy to:

- Build MCP clients that can connect to any MCP server
- Create MCP servers that expose resources, prompts and tools
- Use standard transports like stdio and SSE
- Handle all MCP protocol messages and lifecycle events

## links
- https://github.com/modelcontextprotocol/python-sdk
- https://neo4j.com/blog/developer/knowledge-graphs-claude-neo4j-mcp/
- https://github.com/neo4j-contrib/mcp-neo4j/tree/main
- https://pypi.org/project/mcp-neo4j-cypher/
- 
# CLaude Desktop via MCP

### Install 
https://claude.ai/download

```
linux/mac
curl -LsSf https://astral.sh/uv/install.sh | sh

windows
powershell -ExecutionPolicy ByPass -c "irm https://astral.sh/uv/install.ps1 | iex"

"neo4j": {
    "command": "/Users/<username>/.local/bin/uvx", # full path to your 'uv' executable
    "args": [
        "mcp-neo4j-cypher",
        "--db-url",
        "bolt://localhost",
        "--username",
        "neo4j",
        "--password",
        "password"
    ]
}
```


In [ ]:
from langchain_neo4j import Neo4jGraph, GraphCypherQAChain
from langchain_openai import ChatOpenAI

In [ ]:
import logging
import warnings
from dotenv import load_dotenv
from dotenv import dotenv_values
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
import pprint

In [ ]:
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_neo4j import Neo4jVector
from langchain_openai import OpenAIEmbeddings

In [ ]:
status = load_dotenv(".env")
config = dotenv_values(".env")

In [ ]:
llm = ChatOpenAI(
    model="gpt-4.1",
    openai_api_key=config["OPENAI_API_KEY"],
    temperature=0,
    max_tokens=8192,
)
embeddings = OpenAIEmbeddings()

In [ ]:
uri = "bolt://127.0.0.1:7687"
username = "neo4j"
password = ""

# Neo4jVector
The Neo4j Vector integration supports a number of operations

- create vector from langchain documents
- query vector
- query vector with additional graph retrieval Cypher query
- construct vector instance from existing graph data
- hybrid search
- metadata filtering

In [ ]:
loader = TextLoader("state_of_the_union.txt", encoding="utf-8")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [ ]:
# documents[0]

In [ ]:
docs[0], len(docs)

In [ ]:
db = Neo4jVector.from_documents(
    docs, embeddings, url=uri, username=username, password=password
)

query = "What did the president say about Ketanji Brown Jackson"
docs_with_score = db.similarity_search_with_score(query, k=2)

In [ ]:
docs_with_score

# Neo4j Graph
The Neo4j Graph integration is a wrapper for the Neo4j Python driver. It allows querying and updating the Neo4j database in a simplified manner from LangChain. Many integrations allow you to use the Neo4j Graph as a source of data for LangChain.

In [ ]:
graph = Neo4jGraph(url=uri, username=username, password=password)

In [ ]:
#  Neo4jGraph
query = """
MATCH (v:Visitor_this_year {BadgeId:$badgeID})
WITH v.JobTitle AS jt, v.job_role AS jr, v.what_type_does_your_practice_specialise_in AS spec
MATCH (v2:Visitor_this_year)
WHERE v2.assist_year_before = "1"
  AND (
    toLower(v2.JobTitle) = toLower(jt) OR
    toLower(v2.job_role) = toLower(jr) OR
    toLower(v2.what_type_does_your_practice_specialise_in) = toLower(spec)
  )
WITH v2
LIMIT 3
OPTIONAL MATCH (v2)-[:Same_Visitor]->(vlva:Visitor_last_year_lva)
OPTIONAL MATCH (v2)-[:Same_Visitor]->(vbva:Visitor_last_year_bva)
OPTIONAL MATCH (vlva)-[:attended_session]->(s1:Sessions_past_year)
OPTIONAL MATCH (vbva)-[:attended_session]->(s2:Sessions_past_year)
RETURN v2.BadgeId AS this_year_BadgeId, collect(DISTINCT s1) + collect(DISTINCT s2) AS sessions_last_year
"""

In [ ]:
resp = graph.query(query, params={"badgeID": "VPH826J"})

In [ ]:
# resp

# CypherQAChain
The CypherQAChain is a LangChain component that allows you to interact with a Neo4j graph database in natural language. Using an LLM and the graph schema it translates the user question into a Cypher query, executes it against the graph and uses the returned context information and the original question with a second LLM to generate a natural language response.

In [ ]:
chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
    return_intermediate_steps=True,
)

In [ ]:
chain.invoke(
    "How many Sessions this year have a connection Has_stream to the Stream nursing?"
)

In [ ]:
response = chain.invoke(
    "find all the Visitors this year which  has as job title 'vet/vet surgeon' and  have been in the bva show last year?"
)

In [ ]:
pprint.pprint(response.get("result"))

In [ ]:
# 764KTZS
response = chain.invoke(
    " find for  the Visitor this year with batchID F5C5FRF find 5 sessions this year which are in one of the stream connected using the  relationship job_to_stream"
)

In [ ]:
pprint.pprint(response.get("result"))

In [ ]:
response = chain.invoke(
    """
find for  the Visitor this year with batchID F5C5FRF find 10 sessions this year which are in one of the stream connected using the  relationship job_to_stream,
from the resulting sessions exclude those which contains Equine in the attribute stream
Bear in mind that the attribute stream in sessions is a string that can contains multiple Streams separated by ; ex. 'Orthopaedics; Orthopaedics; Orthopaedics; Small Animal; Small Animal; Small Animal' """
)

In [ ]:
pprint.pprint(response.get("result"))

In [ ]:
response = chain.invoke(
    """
for batchID UEXNXI4 
find 3 visitors this year which assist the year before is 1  with  equal or similar: JobTitle,  job_role and what_type_does_your_practice_specialise_in
and for the 3 visitors you find return all sessions they visited last year and the batchID this year

 """
)

In [ ]:
print(response.get("result"))

In [ ]:
pprint.pprint(response.get("intermediate_steps")[1]["context"])